In [1]:
import sys

#sys.path.append("/home/tayral/Work/TRIQS/source/ed/python")
sys.path.append("../python")
from sympy import *
from sympy.matrices import *
from sympy import init_printing
from sympy.utilities.lambdify import lambdify
init_printing() 
from init_creation_ops import *
from count_bits import *
import itertools

In [2]:
import numpy as np


In [3]:

class hubbard_dimer:
    def __init__(self):
        Norb=4
        Cdags=init_creation_ops(Norb)
        #print Cdags
        U, V, mu, t, w, beta, xi, alpha=symbols('U V mu t omega beta xi alpha')
        C_dag_up_1, C_dag_dn_1, C_dag_up_2, C_dag_dn_2  = Matrix(Cdags[0]),  Matrix(Cdags[1]), Matrix(Cdags[2]), Matrix(Cdags[3]) 

        C_up_1, C_dn_1, C_up_2, C_dn_2  = C_dag_up_1.T, C_dag_dn_1.T, C_dag_up_2.T, C_dag_dn_2.T
        n_up_1 = C_dag_up_1 * C_up_1
        n_dn_1 = C_dag_dn_1 * C_dn_1
        n_up_2 = C_dag_up_2 * C_up_2
        n_dn_2 = C_dag_dn_2 * C_dn_2
        n_1 = n_up_1+n_dn_1
        n_2 = n_up_2+n_dn_2
        N=n_up_1+n_dn_1+n_up_2+n_dn_2
        
        H_2site = xi* (n_up_1*n_dn_1+n_up_2*n_dn_2)   \
                  - (mu-alpha+xi*alpha/U)*N\
                  -t *(C_dag_up_1*C_up_2 + C_dag_up_2*C_up_1 + C_dag_dn_1*C_dn_2 + C_dag_dn_2*C_dn_1)

        def T(i,j):
            '''matrix exchanging rows/cols i and j'''
            T_ij = eye(2**Norb)
            T_ij[i,i]=0
            T_ij[j,j]=0
            T_ij[i,j]=1
            T_ij[j,i]=1
            return T_ij

        def diagonalize_blocks(M_block):
            L_P = [] #list of eigenvects
            L_D = [] #list of diags
            for B in M_block.blocks:
                P,D = B.diagonalize(normalize=True)
                L_P.append(P)
                L_D.append(D)
            P = BlockMatrix([L_P])
            D = BlockMatrix([L_D])
            return P,D
        def from_blockMatrix_to_Matrix(M):
            '''assuming square matrix'''
            s = sum([B.shape[0] for B in M.blocks])
            Mfull = Matrix(zeros((s,s)))
            cum=0
            for B in M.blocks:
                Mfull[cum:cum+B.shape[0],cum:cum+B.shape[1]]=B
                cum+= B.shape[0]
            return Mfull

        S = T(3,8)*T(7,12)
        
        H_tilde = S*H_2site*S.T
        H_by_blocks = BlockMatrix([H_tilde.get_diag_blocks()])
        P,eps = diagonalize_blocks(H_by_blocks)
        P_simpl = simplify(P)
        P2 = from_blockMatrix_to_Matrix(P_simpl)
        eps2 = from_blockMatrix_to_Matrix(eps)
        
        Z=sum([exp(- beta*eps2[i,i]) for i in range(2**Norb)])
        G_term = lambda i, j, c1, cd2 : (P2[:,i].T *S* c1*S.T * P2[:,j])*(P2[:,j].T *S* cd2*S.T * P2[:,i])\
                          *(exp(- beta*eps2[i,i])+exp(- beta*eps2[j,j]))/(w + eps2[i,i]-eps2[j,j])
        G_11=sum([G_term(i,j,C_up_1, C_dag_up_1)[0] for (i,j) in itertools.product(range(2**Norb),range(2**Norb))])
        G_12=sum([G_term(i,j,C_up_1, C_dag_up_2)[0] for (i,j) in itertools.product(range(2**Norb),range(2**Norb))])
        G_22=sum([G_term(i,j,C_up_2, C_dag_up_2)[0] for (i,j) in itertools.product(range(2**Norb),range(2**Norb))])
        G_21=sum([G_term(i,j,C_up_2, C_dag_up_1)[0] for (i,j) in itertools.product(range(2**Norb),range(2**Norb))])
        det = G_11*G_22-G_12*G_21
        G_inv = Z*1/det * Matrix(2,2,[G_22,-G_21,-G_21,G_11])
        G_0inv = Matrix(2,2,[w+mu, 0 , 0 ,w+mu])
        Sigma = G_0inv -G_inv
        self.Sigma_lambda_11 = lambdify((xi,alpha,U,t,beta,mu,w),Sigma[0,0], "numpy")
        self.Sigma_lambda_12 = lambdify((xi,alpha,U,t,beta,mu,w),Sigma[0,1], "numpy")
        self.Z_lambda = lambdify((xi,U,mu,t),Z,"numpy")
        self.evaluate11 = lambda z,alpha,U,t_eval,b_eval,mu_eval,i: self.Sigma_lambda_11(z,alpha,U,t_eval,b_eval,mu_eval,1j*np.pi/b_eval*(2*i+1))
        self.evaluate12 = lambda z,alpha,U,t_eval,b_eval,mu_eval,i: self.Sigma_lambda_12(z,alpha,U,t_eval,b_eval,mu_eval,1j*np.pi/b_eval*(2*i+1))

    def plotFig11(alph, U, t, beta, mu, i, x_min, x_max, y_min, y_max, N, r_maxi):
      T = np.zeros((N,N),dtype='complex')
      axis_x = np.linspace(x_min,x_max,N)
      axis_y = np.linspace(y_min,y_max,N)
      X,Y = np.meshgrid(axis_x,axis_y)
      Z = X + Y*1j
      T = self.evaluate11(Z, alph,U,t,beta,mu,i)
      fig = plt.figure(figsize=(10,8))
      plt.imshow(complex_array_to_rgb(T, rmax=r_maxi), extent=(x_min,x_max,y_min,y_max))
      fig.gca().add_artist(patches.Circle((0,0), radius=1, color='b', fill=False, linestyle='dashed', lw=2))
      fig.gca().add_artist(patches.Circle((0,0), radius=U, color='w', fill=False, linestyle='dashed', lw=2))
      fig.suptitle("Dimer plot of Sigma11(xi, omega"+str(i)+"), to be evaluated at xi=U="+str(U)+", with alpha="+str(alph)+", beta="+str(beta)+", mu="+str(mu))

    def plotFig12(alph, U, t, beta, mu, i, x_min, x_max, y_min, y_max, N, r_maxi):
      T = np.zeros((N,N),dtype='complex')
      axis_x = np.linspace(x_min,x_max,N)
      axis_y = np.linspace(y_min,y_max,N)
      X,Y = np.meshgrid(axis_x,axis_y)
      Z = X + Y*1j
      T = self.evaluate12(Z, alph,U,t,beta,mu,i)
      fig = plt.figure(figsize=(10,8))
      plt.imshow(complex_array_to_rgb(T, rmax=r_maxi), extent=(x_min,x_max,y_min,y_max))
      fig.gca().add_artist(patches.Circle((0,0), radius=1, color='b', fill=False, linestyle='dashed', lw=2))
      fig.gca().add_artist(patches.Circle((0,0), radius=U, color='w', fill=False, linestyle='dashed', lw=2))
      fig.suptitle("Dimer plot of Sigma12(xi, omega"+str(i)+"), to be evaluated at xi=U="+str(U)+", with alpha="+str(a)+", beta="+str(beta)+", mu="+str(mu))


In [4]:
%time dimer = hubbard_dimer()

CPU times: user 34 s, sys: 75.2 ms, total: 34 s
Wall time: 34 s


In [5]:
dimer.evaluate11(1+1j,0,1,1,3,1,0)

(1.2033012647221319-2.4370830028679213j)

In [6]:
beta = 3 
U =1.2
mu = 0.48
t=0.5

In [7]:
dimer.plotFig11(0,U,t,beta,mu,0,-6,6,-6,6,500,1.0)

TypeError: plotFig11() takes exactly 12 arguments (13 given)

In [11]:
dimer.plotFig11?